In [1]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy import units as u

from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd

import os

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Rectangle

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  # take environment variables from .env.

%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
filepath = '/Users/mteodoro/IMAGES/ETACAR/MAST_2022-07-13T1256/HST/ie2w04020'
filename = 'ie2w04020_drz.fits'
path = os.path.join(filepath, filename)

hdu = fits.open(path)
wcs_in = WCS(hdu['SCI'].header)

$\eta$ Car's ICRS coordinates from SIMBAD:

In [ ]:
etc_coords = SkyCoord('10h45m03.5377808928s -59d41m04.051599720s', frame='icrs')

Determine the optimal WCS for the image (by default the final WCS will be oriented North-Up East-Left)

In [ ]:
wcs_out, shape_out = find_optimal_celestial_wcs([hdu['SCI']], reference=etc_coords)

Apply the optimal WCS transformations to the original image and save the result

In [ ]:
array, footprint = reproject_and_coadd([hdu['SCI']], wcs_out, shape_out=shape_out, reproject_function=reproject_interp)

In [ ]:
plt.figure(figsize=(18, 8))

vmin = 1
vmax = 100
cmap = 'viridis'
norm = colors.LogNorm(vmin, vmax, clip=True)

ax1 = plt.subplot(1, 2, 1, projection=wcs_in)
im1 = ax1.imshow(hdu['SCI'].data, origin='lower', cmap=cmap, norm=norm)
ax1.set_xlabel('alpha')
ax1.set_ylabel('delta')
ax1.set_title('Original')
ax1.grid(color='white', ls='-', alpha=0.7)

ax2 = plt.subplot(1, 2, 2, projection=wcs_out)
im2 = ax2.imshow(array, origin='lower', cmap=cmap, norm=norm)
ax2.set_xlabel('alpha')
ax2.set_ylabel('delta')
ax2.set_title('NUEL Oriented')
ax2.grid(color='white', ls='-', alpha=0.7)

In [ ]:
plt.figure(figsize=(20, 10))

# size of the cutout image (relative to the central source coordinates)
size = u.Quantity((4, 4), u.arcsec)
cutout = Cutout2D(array, etc_coords, size, wcs=wcs_out)
wcs_cutout = cutout.wcs

vmin = 10
vmax = 10000
cmap = 'viridis'
norm = colors.LogNorm(vmin, vmax)

ax = plt.subplot(projection=wcs_cutout)
ax.set_xlabel('alpha')
ax.set_ylabel('delta')
ax.set_title('Cutout')
# ax.grid(color='white', ls='-', alpha=0.25)

ax.plot_coord(etc_coords, '+', color='red')

im = ax.imshow(cutout.data, origin='lower', cmap=cmap, norm=norm)